In [1]:
import json
from pprint import pprint
import os
import sqlite3
from __future__ import print_function
globalString = ""

## Create functions to read data

In [2]:
def readFolder(path):
    dataPoints = []
    count = 0
    for (dirpath, dirnames, filenames) in os.walk(path):
        # read all files
        for filename in filenames:
            with open(path + filename) as f:
                for line in f:
                    dataPoints.append(json.loads(line))
                    count += 1
    print(str(count) + " data points were read!")
    return dataPoints

def addDataPointToDB(cursor, dataPoint):
    meta = dataPoint["data"]["meta"]
    metadata = dataPoint["metadata"]
    addItems = (meta["source"], meta["input_source"], meta["destination"],
                meta["tool_name"], meta["input_destination"], meta["measurement_agent"], 
                str(dataPoint["data"]["datapoints"]), 
                metadata["topic"], metadata["type"], metadata["kafka_timestamp"], 
                metadata["summaries"], metadata["type_prefix"], metadata["event-type"])
    cursor.execute("INSERT INTO rawdata VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?)", addItems)

def readDataFromFoderToDB(path, cursor, conn):
    count = 0
    printSteps = 10000
    print("Currently read data points: ")
    for (dirpath, dirnames, filenames) in os.walk(path):
        # read all files
        for filename in filenames:
            with open(path + filename) as f:
                for line in f:
                    global globalString
                    globalString = line
                    if(line == ""):
                        break;
                    try:
                        dataPoint = json.loads(line)
                    except:
                        print("The json parser just had an exception, I would guess the file is somewhat broken:")
                        print("We will continue to read")
                    addDataPointToDB(cursor, dataPoint)
                    count += 1
                    if (count % printSteps == 0):
                        print(str(count) + ', ', end="")
                        conn.commit()
    print("")
    print("Total read data points: " + str(count))
    

## Connect to DB

In [3]:
dataFolder = "/root/jupyter_NB/raw_data/hadoop/packetloss-08/"
pathToDB = "/root/jupyter_NB/packetloss-DB-1.db"
conn = sqlite3.connect(pathToDB)
cursor = conn.cursor()

## initilize DB

In [4]:
tableCreationString = '''CREATE TABLE rawdata (source text, input_source text, 
destination text, tool_name text, input_destination text, 
measurement_agent  text, datapoints text, topic text, type text, event_timestamp integer, 
summaries text, type_prefix text , event_type text )'''

# check if table already exists if not create it
cursor.execute("SELECT name FROM sqlite_master WHERE type='table' AND name='rawdata';")
data =  cursor.fetchall()
if data.count(("rawdata",)) == 0:
    cursor.execute(tableCreationString)
conn.commit()

## read Data into DB

In [5]:
readDataFromFoderToDB(dataFolder, cursor, conn)
conn.commit()
conn.close()

Currently read data points: 
10000, 20000, 30000, 40000, 50000, 60000, 70000, 80000, 90000, 100000, 110000, 120000, 130000, 140000, 150000, 160000, 170000, 180000, 190000, 200000, 210000, 220000, 230000, 240000, 250000, 260000, 270000, 280000, 290000, 300000, 310000, 320000, 330000, 340000, 350000, 360000, 370000, 380000, 390000, 400000, 410000, 420000, 430000, 440000, 450000, 460000, 470000, 480000, 490000, 500000, 510000, 520000, 530000, 540000, 550000, 560000, 570000, 580000, 590000, 600000, 610000, 620000, 630000, 640000, 650000, 660000, 670000, 680000, 690000, 700000, 710000, 720000, 730000, 740000, 750000, 760000, 770000, 780000, 790000, 800000, 810000, 820000, 830000, 840000, 850000, 860000, 870000, 880000, 890000, 900000, 910000, 920000, 930000, 940000, 950000, 960000, 970000, 980000, 990000, 1000000, 1010000, 1020000, 1030000, 1040000, 1050000, 1060000, 1070000, 1080000, 1090000, 1100000, 1110000, 1120000, 1130000, 1140000, 1150000, 1160000, 1170000, 1180000, 1190000, 1200000,

In [ ]:
print(globalString)